**STACK INSTRUCTIONS** 
- R6 will store "going" addresses while going to push/pop
- R7 will store "return" addresses while going to push/pop
- R5 will be used in push/pop to point to top of stack, so won't be saved

To call a subroutine, using push directly: 
-     LEA R6, [SUBROUTINE LABEL]
-     JSR PUSH

Before returning from a subroutine, using pop directly: 
-     LEA R0, POP
-     JMP R0

To call a subroutine, using push indirectly (NOTE: modifies R1, save or use a different register if necessary): 
-      LEA R6, [SUBROUTINE LABEL]
-      LD R1, [LABEL OF MEMORY CONTAINING .FILL PUSH]
-      JSRR R1
        
To return from a subroutine, using pop indirectly:
-      LD R0, [LABEL OF MEMORY CONTAINING .FILL POP]
-      JMP R0

Path: 
- Store address of subroutine in R6
- JSR to push, which stores starting point in R7
- Push does JMP R6 to get to subroutine
- Subroutine JMPs to pop
- Pop RETs to starting point 

Eventually, we may need to alter some of these instructions as things get too far apart for JSR or w/e.

**ROOM/STATE DATA FORMAT**

(subject to changes?)

Every room/state stored in memory has a label pointing to the beginning of its data block, and variables stored at regular distances relative to that top. 

For all rooms:
- Label + 0: which quality (HP/stats) the room changes (0 for none) [Note: Could this field also be used to change variables, e.g. whether you have a sword?]
- Label + 1: amount of quality change
- Label + 2: type of state (0 for user-input directed, 1 for dice-directed, 2 for variable-directed, 3 for game end, 4 for simple message room)

Then if user-input room: 
- Label + 3, 4, 5: addresses of destinations for 1, 2, and 3 inputs
- Label + 6: Description string in compressed format

Or if dice room: 
- Label + 3: Stat to check
- Label + 4: Difficulty of check
- Label + 5: Destination if success
- Label + 6: Destination if failure

Or if variable room:
- Label + 3: Address of variable to check 
- Label + 4: Value of that variable that counts as a success
- Label + 5: Destination if success
- Label + 6: Destination if failure

Or if end room: 
- Label + 3: Endgame message

Or if simple message room: 
- Label + 3: Unconditional destination
- Label + 4: Message to display

**STAT & VARIABLE CODES**
1. HP
2. CHA
3. VIT
4. WIS
5. SWORD

In [377]:
.ORIG x3000
BRnzp START

;;-----------------------------------------------------------------
;; Setup for the stack 
Top:    .FILL Stack
Stack:  .BLKW #100

PUSH:   LD R5, Top
        STR R0, R5, #0      ;; Save R0 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R1, R5, #0      ;; Save R1 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R2, R5, #0      ;; Save R2 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R3, R5, #0      ;; Save R3 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R4, R5, #0      ;; Save R4 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R6, R5, #0      ;; Save R6 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        STR R7, R5, #0      ;; Save R7 to Stack
        ADD R5, R5, #1      ;; Increment Stack
        ST R5, Top
        JMP R6
        
POP:    LD R5, Top
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R7, R5, #0      ;; Load stack into R7
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R6, R5, #0      ;; Load Stack into R6
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R4, R5, #0      ;; Load Stack into R4
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R3, R5, #0      ;; Load Stack into R3
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R2, R5, #0      ;; Load Stack into R2
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R1, R5, #0      ;; Load Stack into R1
        ADD R5, R5, #-1     ;; Decrement Stack
        LDR R0, R5, #0      ;; Load Stack into R0
        ST R5, Top
        RET

;;-----------------------------------------------------------------
;; Setup, name and class choice

;; Room where user chooses their class year
;; Needs four choices and does not follow main game loop
;; Based off code in Printing_text (NOT Room Structure Demo)

START:    LEA R0, CC_TEXT ;; loads & prints room text
          PUTSP
          IN ;; gets user input
          
          LEA CC_CHOICE, R6  ;; Carries out class choice in subroutine
          JSR PUSH
          
          LDI R0, INITIAL_IND1 ;; Load initial location into LOC
          STI R0, LOC_IND1

          LD R0, LOOP_START_IND
          JMP R0
          
;; Signposts
LOOP_START_IND: .FILL LOOP_START
INITIAL_IND1: .FILL INITIAL_LOC
LOC_IND1: .FILL LOC
VIT_IND1: .FILL VIT
CHA_IND1: .FILL CHA
WIS_IND1: .FILL WIS

;; Subroutine for parsing input
;; Convert the ASCII value in R0 to actual hex value by subtracting 30
;; Subtract 2 from the result
;; 1-2 = -1 so if result is negative, the user input 1
;; 2-2 = 0 so if result is zero, the user input 2
;; 3-2 = 1 so if result is positive, the user input 3 or 4
CC_CHOICE:   
          LD R1, CC_ASCII     ;; Load inverted ASCII conversion constant into R1
          AND R2, R2, #0   ;; Clear R2
          ADD R0, R0, R1   ;; Add ASCII constant to convert ASCII value into a number
          ADD R0, R0, #-2
          BRn CC_CH1 ;; user chose 1
          BRz CC_CH2 ;; user chose 2
          BRp CC_CH34 ;; user chose 3 or 4
          
CC_CH1:      LEA R0, CC_TEXT1
          ADD R2, R2, #1
          ST R2, CLASS ;; sets CLASS to 1
          ST R7, CC_SAVE_R7
          PUTSP
          LD R7, CC_SAVE_R7 
          AND R3, R3, #0   ;; Set vitality to 2
          ADD R3, R3, #2
          STI R3, VIT_IND1
          LEA R0, POP
          JMP R0
CC_CH2:      LEA R0, CC_TEXT2
          ADD R2, R2, #2
          ST R2, CLASS ;; sets CLASS to 2
          ST R7, CC_SAVE_R7
          PUTSP
          LD R7, CC_SAVE_R7
          AND R3, R3, #0   ;; Set vitality to 1
          ADD R3, R3, #1
          STI R3, VIT_IND1
          AND R3, R3, #0   ;; Set charisma to 1
          ADD R3, R3, #1
          STI R3, CHA_IND1
          LEA R0, POP
          JMP R0
CC_CH34:     ADD R0, R0, #-2
          BRn CC_CH3 ;; user chose 3
          BRzp CC_CH4 ;; user chose 4
CC_CH3:      LEA R0, CC_TEXT3
          ADD R2, R2, #3
          ST R2, CLASS ;; sets CLASS to 3
          ST R7, CC_SAVE_R7
          PUTSP
          LD R7, CC_SAVE_R7
          AND R3, R3, #0   ;; Set charisma to 1
          ADD R3, R3, #1
          STI R3, CHA_IND1
          AND R3, R3, #0   ;; Set wisdom to 1
          ADD R3, R3, #1
          STI R3, WIS_IND1
          LEA R0, POP
          JMP R0
CC_CH4:      LEA R0, CC_TEXT4
          ADD R2, R2, #4
          ST R2, CLASS ;; sets CLASS to 4
          ST R7, CC_SAVE_R7
          PUTSP
          LD R7, CC_SAVE_R7
          AND R3, R3, #0   ;; Set wisdom to 2
          ADD R3, R3, #2
          STI R3, WIS_IND1
          LEA R0, POP
          JMP R0

;; To save R7 so PUTSP doesn't overwrite return path
CC_SAVE_R7: .FILL #0
;;;--------------------------------------------------------------------------------------
;; Data for class choice subroutine
CC_ASCII:   .FILL x-30
CLASS:   .FILL #0
CC_TEXT1:   .STRINGC "You are a freshman! +2 Vitality\n"
CC_TEXT2:   .STRINGC "You are a sophomore! +1 Vitality, +1 Charisma\n"
CC_TEXT3:   .STRINGC "You are a junior! +1 Charisma, +1 Wisdom\n"
CC_TEXT4:   .STRINGC "You are a senior! +2 Wisdom\n"
CC_TEXT:    .STRINGC "It's the start of a new day at Bryn Mawr College! What class year are you?\nIn order to choose a year, input the number next to your choice. (Note that if you input an invalid option, it will be interpreted as one of the given options.)\n1  Be a freshman.\n2  Be a sophomore.\n3  Be a junior.\n4  Be a senior.\n" 

;; Game status message text data
STAT_CAP_INITIALS: .FILL #0
CAP_TREF: .FILL CAPITAL_T
CAP_CREF: .FILL CAPITAL_C
CAP_VREF: .FILL CAPITAL_V
CAP_WREF: .FILL CAPITAL_W

CAPITAL_T: .STRINGC "T"
CAPITAL_C: .STRINGC "C" 
CAPITAL_V: .STRINGC "V"
CAPITAL_W: .STRINGC "W" 

STAT_INITIALS: .FILL #0
TREF: .FILL T
CREF: .FILL C
VREF: .FILL V
WREF: .FILL W

T: .STRINGC "t"
C: .STRINGC "c"
V: .STRINGC "v" 
W: .STRINGC "w"

STAT_NAMES: .FILL #0 ;; Start of stat names block
HP_NAMEREF: .FILL HP_NAME
CHA_NAMEREF: .FILL CHA_NAME
VIT_NAMEREF: .FILL VIT_NAME
WIS_NAMEREF: .FILL WIS_NAME

HP_NAME: .STRINGC "ime remaining"
CHA_NAME: .STRINGC "harisma"
VIT_NAME: .STRINGC "itality"
WIS_NAME: .STRINGC "isdom"

STAT_T1: .STRINGC "Your "
STAT_T2: .STRINGC " increased by "
STAT_T3: .STRINGC " decreased by "
STAT_T4: .STRINGC "!\n"

DICE_T1: .STRINGC " check: You rolled "
DICE_T2: .STRINGC " with a stat bonus of " 
DICE_T3: .STRINGC " and succeeded!\n"
DICE_T4: .STRINGC " and failed!\n"

STATS_ADDR: .FILL STATS
STATNAMES_ADDR: .FILL STAT_NAMES

;; Variables for main loop
INITIAL_LOC: .FILL ROOM_UINPUT
OUT_OF_TIME: .FILL ENDGAME
LOC: .FILL ROOM_UINPUT  ;; NOTE: Replace this with actual starting room
NAME: .STRINGC "Ruby"   ;; NOTE: Replace this with user-input name
STATS: .FILL #0 ;; Start of stats block
HP: .FILL #10    ;; Stats + 1
CHA: .FILL #0   ;; Stats + 2
VIT: .FILL #0   ;; Stats + 3
WIS: .FILL #0   ;; Stats + 4
SWD: .FILL #0   ;; Stats + 5
;; -----------------------------------------------------------
;; Main game loop
LOOP_START: 

    LD R0, HP
    BRp HP_LEFT
    
    ;; If user is out of time, proceed to "endgame" section
    LD R0, OUT_OF_TIME
    ST R0, LOC

    ;; Todo: Print name and stylized HP - maybe only if in uinput room? 

HP_LEFT: 
    LD R1, LOC     ;; Load current state's address into R1 so we can access variables relative to it

    ;; Perform stat increase for room, if it has one
    
    LDR R0, R1, #0 ;; Load which_quality into R0
    BRz NO_CHANGE  ;; If which_quality is zero, skip stat change

    LD R5, STATS_ADDR ;; Load the address of the stat array into R5
    ADD R2, R0, R5 ;; Add the offset in R0 to the address in R5 to get the address of the appropriate stat
    LDR R3, R2, #0 ;; Load the current value of that stat into R3
    LDR R4, R1, #1 ;; Load R4 with the amount the stat will be changed
    ADD R3, R3, R4 
    STR R3, R2, #0 ;; Store modified stat back into memory
    
    ;; Print message about stat change
    ;; ONLY if stat code is 1-4 - otherwise it's a variable
    ;; and shouldn't be announced (ex., no message for "SWORD increased")
    
    LDR R0, R1, #0 
    ADD R0, R0, #-5
    BRzp NO_CHANGE ;; If which_quality is >4, don't print
    
    LEA R0, STAT_T1
    PUTSP
    
    LDR R2, R1, #0
    
    LEA R3, STAT_INITIALS
    ADD R3, R3, R2
    LDR R0, R3, #0
    PUTSP
    
    LEA R3, STAT_NAMES
    ADD R3, R3, R2
    LDR R0, R3, #0
    PUTSP

    ;; Print "increased by .. " message if stat change was positive and "decreased" otherwise

    ADD R4, R4, #0
    BRn STAT_NEG
    
    LEA R0, STAT_T2
    PUTSP
    
    ST R4, CONVERT_VAL
    LEA R6, ASCII_CONVERT
    JSR PUSH
    
    LEA R0, STAT_T4
    PUTSP
    
    BRnzp NO_CHANGE
    
STAT_NEG:    
    LEA R0, STAT_T3
    PUTSP
    
    NOT R4, R4
    ADD R4, R4, #1 ;; First negate the stat, then print
    ST R4, CONVERT_VAL
    LEA R6, ASCII_CONVERT
    JSR PUSH
    
    LEA R0, STAT_T4
    PUTSP
    
NO_CHANGE:
    LDR R2, R1, #2 ;; Load room_type into R2
    LEA R3, ROOMTYPES ;; Load pointer to ROOMTYPES array into R3
    ADD R3, R3, #1    
    ADD R2, R2, R3 ;; Add room_type to ROOMTYPES to get a reference to the correct location
    LDR R2, R2, #0
    JMP R2
    
ROOMTYPES: .FILL #0
UNINPUT_REF: .FILL UINPUT
DICE_REF: .FILL DICE
VAR_REF: .FILL VARIABLE
END_REF: .FILL ENDING
MESSAGE_REF: .FILL MESSAGE

UINPUT:            
    ;; For user-directed rooms, print description, then use subroutine that reads, parses and applies number from 1-3
    ADD R0, R1, #6 ;; Load R0 with address of room's description
    PUTSP          ;; Print room description
    
    LEA R6, CHOICE
    JSR PUSH
    BRnzp LOOP_DONE

DICE: 
    ;; Get a roll using D16
    LEA R6, D16
    JSR PUSH
    ;; Now there is a number between 1 and 16 in D16_roll
    
    ;; Load which_stat into R2
    LDR R2, R1, #3
    
    ;; Print name of stat
    LEA R0, STAT_CAP_INITIALS
    ADD R0, R0, R2
    LDR R0, R0, #0
    PUTSP
    
    LEA R0, STAT_NAMES
    ADD R0, R0, R2
    LDR R0, R0, #0
    PUTSP
    
    ;; Print dice text
    LEA R0, DICE_T1
    PUTSP
    
    LD R0, D16_roll
    
    ;; Print dice message
    ST R0, CONVERT_VAL
    LEA R6, ASCII_CONVERT
    JSR PUSH
    
    LEA R0, DICE_T2
    PUTSP
    
    ;; Get the value of that stat
    LEA R3, STATS
    ADD R3, R3, R2
    LDR R3, R3, #0
    
    ;; Print stat value
    ST R3, CONVERT_VAL
    LEA R6, ASCII_CONVERT
    JSR PUSH
    
    LD R0, D16_roll
    
    ;; Add it to the dice roll to get [result] + [stat bonus]
    ADD R0, R0, R3
    
    ;; Load difficulty check value into R2
    LDR R2, R1, #4
    
    ;; Negate R2 for comparison
    NOT R2, R2
    ADD R2, R2, #1
    
    ;; Add R2 to R0 to compare difficulty check to total roll
    ADD R2, R2, R0
    
    ;; Now R2 is negative if failure or zero/positive for success
    BRn FAILURE
    
    ;; If success, print that you succeeded, then load address of success location into LOC
    
    LEA R0, DICE_T3
    PUTSP
    
    LD R2, LOC
    ADD R2, R2, #5
    BRnzp DICE_DONE

FAILURE: 
    LEA R0, DICE_T4
    PUTSP

    LD R2, LOC
    ADD R2, R2, #6

DICE_DONE: 
    LDR R3, R2, #0 ;; Load new location at address R2 into R3
    ST R3, LOC     ;; Store new location in LOC 
    BRnzp LOOP_DONE

VARIABLE: 
    LDR R0, R1, #3 ;; Load VAR_ADDRESS into R0
    LDR R0, R0, #0 ;; Load value of variable at VAR_ADDRESS into R0
    LDR R2, R1, #4 ;; Load min value into R2
    NOT R2, R2     
    ADD R2, R2, #1 ;; Negate R2
    ADD R2, R2, R0 ;; Compare
    BRn VAR_FAILURE
    
    ;; Success
    LD R2, LOC
    ADD R2, R2, #5
    BRnzp VAR_DONE
    
VAR_FAILURE: 
    LD R2, LOC
    ADD R2, R2, #6

VAR_DONE:
    LDR R3, R2, #0 ;; Load new location at address R2 into R3
    ST R3, LOC     ;; Store new location in LOC 
    BRnzp LOOP_DONE
    
ENDING: 
    ADD R0, R1, #3 ;; Load R0 with address of room's description
    PUTSP          ;; Print room description
    HALT           ;; End game (Note: Should this be restart?)
    
MESSAGE: 
    ADD R0, R1, #4 ;; Load R0 with address of room's description
    PUTSP          ;; Print description
    ADD R0, R0, #-1 ;; R0 now has address containing destination 
    LDR R0, R0, #0  
    ST R0, LOC
    BRnzp LOOP_DONE
    
LOOP_DONE: 
    BRnzp LOOP_START   ;; Repeat main game loop
    HALT               ;; Right now this halt is never reached bc it loops forever LMAO

;; ------------------------------------------------------------------
;; Signposts to PUSH and POP

PUSH_IND1: .FILL PUSH
POP_IND1: .FILL POP

;; ------------------------------------------------------------------
;; Subroutine to get a number from 1-3 from the user and move user to corresponding location
;; Copied from Anita's code in printing_text

CHOICE:   
          IN
          LD R2, LOC
          LD R1, ASCII     ;; Load inverted ASCII conversion constant into R1
          ADD R0, R0, R1   ;; Add it to convert ASCII value into a number
          ADD R0, R0, #-2
          BRn CH1 ;; user chose 1
          BRz CH2 ;; user chose 2
          BRp CH3 ;; user chose 3
          
CH1:      ADD R2, R2, #3
          BRnzp CH_DONE
CH2:      ADD R2, R2, #4
          BRnzp CH_DONE
CH3:      ADD R2, R2, #5
          BRnzp CH_DONE
          
CH_DONE:  LDR R3, R2, #0 ;; Load the data at R2, aka the address of the #[choice] room, into R3
          ST LOC, R3     ;; Store that data in LOC, moving player there 
          LD R0, POP_IND1
          JMP R0

;; Data
ASCII:   .FILL x-30

;;; -----------------------------------------------------
;;; After this runs, there is a new number between 1 and 16 in D16_roll
D16: 
    ;; Call Random, loading a new random number into X
    LEA R6, Random
    LD R1, PUSH_IND1
    JSRR R1
    
    LD R0, X
    AND R0, R0, #15  ;; Mod number by 16
    ADD R0, R0, #1   ;; Add one to get number between 1 and 16
    ST R0, D16_roll
    
    LD R0, POP_IND1
    JMP R0
D16_roll: .FILL #0

;; -------------------------------------------------------
;; Subroutine to convert binary value into ASCII decimal and print.
;; Note: Algorithm does not work reliably on 3-digit or larger numbers. 

ASCII_CONVERT:
    LD R0, CONVERT_VAL
    ST R0, TEMPX
    ST R0, TEMPY
    
ASCII_BASE: ;; Base case: if val < #10, add x30 and print

    ;; reset TEMPZ to #-1
    LD R1, NEG_ONE
    ST R1, TEMPZ
    
    ;; check val < #10
    ADD R1, R0, #-10
    BRzp ASCII_LOOP ;; if val >= #10, skip to main loop
    
    ;; convert R1 to ASCII and print
    LD R1, ASCII_POS
    ADD R0, R0, R1
    OUT R0
    
    LD R0, POP_IND1
    JMP R0
    
ASCII_LOOP_REP: ;; start here if repeating non-base case

    ST R2, TEMPY ;; stores TEMPY from last loop

ASCII_LOOP: ;; Non-base case
    
    ;; TEMPZ = z*10
    LD R0, TEMPZ ;; R0 <- z
    LD R1, TEN ;; R1 <- 10
    LEA R6, Multiply
    LD R2, PUSH_IND1
    JSRR R2         ; R0 * R1
    LD R2, MULT_PRODUCT  ; R2 <- R0 * R1   = z*10
    ST R2, TEMPZ
    
    ;; R2 = y/10
    LD R0, TEMPY ;; R0 <- y
    LEA R6, Divide
    LD R2, PUSH_IND1
    JSRR R2         ; R0 / R1
    LD R2, QUOTIENT  ; R2 <- R0 / R1       = y/10
    ;;ST R2, TEMPY
    ;;ADD R2, R2, #0
    
    BRp ASCII_LOOP_REP ;; if quotient is positive, repeat loop
    
    ;; else if quotient is zero, use previous quotient (current val of TEMPY) as y
    
    ;; z = z/10 since the loop has to go one extra time to hit zero
    LD R0, TEMPZ ;; R0 <- z
    LEA R6, Divide
    LD R2, PUSH_IND1
    JSRR R2         ; R0 / R1
    LD R2, QUOTIENT  ; R2 <- R0 / R1    = z/10
    LD R0, QUOTIENT
    
    ;; R2 = y*z
    LD R1, TEMPY ;; R1 <- y
    LEA R6, Multiply
    LD R2, PUSH_IND1
    JSRR R2         ; R0 * R1
    LD R2, MULT_PRODUCT  ; R2 <- R0 * R1     = y*z
    
    LD R3, ASCII_POS
    ADD R0, R1, R3 ;; convert y to ASCII
    OUT R1
    
    LD R0, TEMPX
    ADD R0, R0, R2 ;; x-z: current state of val - (10^n * y)
    ST R0, TEMPX
    ST R0, TEMPY
    
    BRnzp ASCII_BASE ;; return to ASCII_BASE to check for base case
    
ASCII_POS:   .FILL x30
CONVERT_VAL:   .FILL #0 ;; value to be converted; set before calling
NEG_ONE:   .FILL #-1
TEN:   .FILL #10
TEMPX:   .FILL #0 ;; stores present state of value to be converted, after subtraction
TEMPY:   .FILL #0 ;; stores present state of value to be converted, after division
TEMPZ:   .FILL #-1 ;; stores (negative) exponent of 10

;;; -----------------------------------------------------
;;; Memory X has next random number after this runs
Random: 
    LD R0, M
    LD R1, A
    
    LEA R6, Divide
    LD R2, PUSH_IND1
    JSRR R2         ; R0 / R1
    
    ;; q = m / a
    LD R0, QUOTIENT     ; R0 / R1
    ST R0, Q 
    ;; r = m mod a
    LD R0, REMAINDER    ; R0 mod R1
    ST R0, R
        ;; x / q
    LD R0, X
    LD R1, Q
    
    LEA R6, Divide
    LD R2, PUSH_IND1
    JSRR R2         ; R0 / R1
    
    LD R1, QUOTIENT
    ST R1, TEMP2
    LD R1, REMAINDER    ; x mod q
    ST R1, TEMP1
    ;;      a * TEMP1 - r * TEMP2
    LD R0, A
    
    LEA R6, Multiply
    LD R2, PUSH_IND1
    JSRR R2         ; R0 * R1
    LD R2, MULT_PRODUCT  ; R2 <- R0 * R1
    
    ST R2, TEMP1
    ;;      a * TEMP1 - r * TEMP2
    LD R0, R
    LD R1, TEMP2
    
    LEA R6, Multiply
    LD R2, PUSH_IND1
    JSRR R2         ; R0 * R1          
    LD R2, MULT_PRODUCT ; R2 <- r * TEMP2
    
    NOT R2,R2           ; -R2
    ADD R2,R2,#1
    ST R2, TEMP2 
    LD R1, TEMP1
    ADD R2, R2, R1      ; TEMP1 - TEMP2
    
TEST:  BRzp DONE        ; if x < 0 then
    LD R1, M
    ADD R2, R2, R1      ; 
DONE: ST R2, X
    LD R0, POP_IND1         ;; Return
    JMP R0

A: .FILL #7             ;; a , the multiplicative constant is given
M: .FILL #32767         ;; m = 2 Ë† 15 âˆ’ 1, the modulus is given
X: .FILL #10            ;; x, the seed is given
R: .FILL #0
Q: .FILL #0
TEMP1: .FILL #0
TEMP2: .FILL #0
BACK: .FILL #0

;;; -----------------------------------------------------
;;; Mult_Product <- R0 * R1
;;; Also uses R3 to store SIGN
Multiply: AND R2,R2,#0
  AND R3,R3,#0
  ADD R0,R0,#0         ; compare R0
  BRn MultNEG1
  BR  MultCont
MultNEG1: NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
MultCONT: ADD R1,R1,#0         ; compare R1
  BRn MultNEG2
  BR MultInit
MultNEG2: NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
MultInit: ADD R0,R0,#0      ; have R0 set the condition codes
MultLoop: BRz MultDone
  ADD R2,R2,R1
  ADD R0,R0,#-1
  BR MultLoop
MultDone: ADD R0,R3,#0
  BRzp MultRet
  NOT R2,R2
  ADD R2,R2,#1
MultRet:
  ST R2, MULT_PRODUCT           ; Product has the product
  LD R0, POP_IND1
  JMP R0
  
MULT_PRODUCT: .FILL #0

;;; -----------------------------------------------------
;;; R0 / R1
;;; Also uses R3 to store SIGN
;;;           R4 to store -R1
;;;           R5 is QUOTIENT
;;;           R6 is REMAINDER
;;;           R2 temp
Divide:   AND R3,R3,#0
  ST R3, QUOTIENT
  ST R3, REMAINDER
  ADD R0,R0,#0         ; compare R0
  BRn DivNEG1
  BR  DivCont
DivNEG1:  NOT R3,R3         ; flip SIGN
  NOT R0,R0
  ADD R0,R0,#1
DivCONT:  ADD R1,R1,#0         ; compare R1
  BRn DivNEG2
  BR DivInit
DivNEG2:  NOT R3,R3         ; flip SIGN
  NOT R1,R1
  ADD R1,R1,#1
DivInit:  ADD R4,R1,#0
  NOT R4,R4
  ADD R4,R4,#1
DivLoop:  ADD R2,R0,R4      ; have R2 set the condition codes
  BRn DivDone
  ADD R0,R0,R4
  LD R2,QUOTIENT
  ADD R2,R2,#1
  ST R2,QUOTIENT
  BR DivLoop
DivDone:  ADD R3,R3,#0         ; Negative?
  BRzp DivRet
  LD R2,QUOTIENT     ; Yes, then negate R2
  NOT R2,R2
  ADD R2,R2,#1
  ST R2,QUOTIENT
DivRet:      ST R0,REMAINDER
  LD R0, POP_IND1
  JMP R0
QUOTIENT: .FILL #0
REMAINDER: .FILL #0

;; ------------------------------------------------------------------

ROOM_UINPUT: 
    .FILL #0 ;; Which quality (stat/HP) it changes: None
    .FILL #0 ;; Amount of change: None
    .FILL #0 ;; 0: Based on user input
    .FILL ROOM_VARIABLE
    .FILL ROOM_RANDOM
    .FILL SWORD_ROOM
    .STRINGC "Here we are in an example room. Enter 1 to TEST UR SWORD, 2 to go to the random room, or 3 to GET A SWORD\n"
    
ROOM_RANDOM: 
    .FILL #1 ;; Quality changed: HP
    .FILL #-1;; Amount of change: 1
    .FILL #1 ;; 1: Based on dice roll
    .FILL #3 ;; Stat for dice roll: Vitality
    .FILL #15 ;; Difficulty of check
    .FILL ROOM_UINPUT ;; Destination if success
    .FILL ROOM_RANDOM ;; Destination if failure
    
MESSAGE_RM: 
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #4 ;; 4: Message state
    .FILL ROOM_UINPUT
    .STRINGC "You passed the sword check!\n" 
    
SWORD_ROOM:
    .FILL #5 ;; Quality changed: Sword
    .FILL #1 ;; Amount of change: One
    .FILL #0 ;; 0: Based on user input
    .FILL ROOM_UINPUT
    .FILL ROOM_UINPUT
    .FILL ROOM_UINPUT ;; Three choices all lead to user input room
    .STRINGC "SWORD ROOM SWORD ROOM SWORM never come back or u will lose your sword\n"
    
ROOM_VARIABLE:
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #2 ;; 2: Based on variable
    .FILL SWD ;; VAR_ADDRESS ;; What variable to check
    .FILL #1 ;; VALUE       ;; Minimum value that counts as a success
    .FILL MESSAGE_RM ;; Destination if success
    .FILL SWORD_DEATH ;; Destination if failure
    
SWORD_DEATH: 
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #3 ;; 3: End state
    .STRINGC "Your SWORDLESSNESS was no match for the TEST OF SWORDS\n"
    
ENDGAME: ;; Note: doesn't HAVE to be an ending state, could be uinput and lead to others
    .FILL #0 ;; Quality changed: None
    .FILL #0 ;; Amount of change: None
    .FILL #3 ;; 3: End state
    .STRINGC "You ran out of time!\n"
    
.END

Assembled! Use %dis or %dump to examine; use %exe to run.


In [378]:
%exe

It's the start of a new day at Bryn Mawr College! What class year are you?
In order to choose a year, input the number next to your choice. (Note that if you input an invalid option, it will be interpreted as one of the given options.)
1  Be a freshman.
2  Be a sophomore.
3  Be a junior.
4  Be a senior.

Input a character> 2
2
You are a sophomore! +1 Vitality, +1 Charisma
Here we are in an example room. Enter 1 to TEST UR SWORD, 2 to go to the random room, or 3 to GET A SWORD

Input a character> 2
2
Your time remaining decreased by 1!
Vitality check: You rolled 7 with a stat bonus of 1 and failed!
Your time remaining decreased by 1!
Vitality check: You rolled 11 with a stat bonus of 1 and failed!
Your time remaining decreased by 1!
Vitality check: You rolled 7 with a stat bonus of 1 and failed!
Your time remaining decreased by 1!
Vitality check: You rolled 11 with a stat bonus of 1 and failed!
Your time remaining decreased by 1!
Vitality check: You rolled 12 with a stat bonus of 1 and 

In [368]:
%cont

 and succeeded!
Here we are in an example room. Enter 1 to TEST UR SWORD, 2 to go to the random room, or 3 to GET A SWORD

Input a character> 

Keyboard Interrupt!


In [366]:
%bp 327E

Breakpoints
    1)            x325A: x1920  ADD R4, R4, #0                            [line: 264]
    2) DICE:      x327E: xEC59  LEA R6, D16                               [line: 318]
    3)            x3290: xF024  PUTSP                                     [line: 348]
    4)            x32E0: xC000  JMP R0                                    [line: 486]
    5)            x32ED: xC000  JMP R0                                    [line: 514]
    6) DONE:      x333C: x3404  ST R2, X                                  [line: 633]
    7) DIVDONE:   x337B: x16E0  ADD R3, R3, #0                            [line: 711]


In [176]:
%bp 337B

Breakpoints
    1) DICE:      x327E: xEC59  LEA R6, D16                               [line: 318]
    2) DIVDONE:   x337B: x16E0  ADD R3, R3, #0                            [line: 711]


In [200]:
%bp 333C

Breakpoints
    1) DICE:      x327E: xEC59  LEA R6, D16                               [line: 318]
    2) DONE:      x333C: x3404  ST R2, X                                  [line: 633]
    3) DIVDONE:   x337B: x16E0  ADD R3, R3, #0                            [line: 711]


In [235]:
%bp 32E0

Breakpoints
    1) DICE:      x327E: xEC59  LEA R6, D16                               [line: 318]
    2)            x32E0: xC000  JMP R0                                    [line: 486]
    3) DONE:      x333C: x3404  ST R2, X                                  [line: 633]
    4) DIVDONE:   x337B: x16E0  ADD R3, R3, #0                            [line: 711]


In [260]:
%bp 325A

Breakpoints
    1)            x325A: x1920  ADD R4, R4, #0                            [line: 264]
    2) DICE:      x327E: xEC59  LEA R6, D16                               [line: 318]
    3)            x32E0: xC000  JMP R0                                    [line: 486]
    4) DONE:      x333C: x3404  ST R2, X                                  [line: 633]
    5) DIVDONE:   x337B: x16E0  ADD R3, R3, #0                            [line: 711]


In [336]:
%bp 32ED

Breakpoints
    1)            x325A: x1920  ADD R4, R4, #0                            [line: 264]
    2) DICE:      x327E: xEC59  LEA R6, D16                               [line: 318]
    3)            x32E0: xC000  JMP R0                                    [line: 486]
    4)            x32ED: xC000  JMP R0                                    [line: 514]
    5) DONE:      x333C: x3404  ST R2, X                                  [line: 633]
    6) DIVDONE:   x337B: x16E0  ADD R3, R3, #0                            [line: 711]


In [371]:
%bp 3294

Breakpoints
    1)            x3294: x267B  LD R3, CONVERT_VAL                        [line: 356]


In [30]:
%dis 3256

Memory disassembled:
           x3256: xE781  LEA R3, STAT_NAMES                        [line: 257]
           x3257: x16C2  ADD R3, R3, R2                            [line: 258]
           x3258: x60C0  LDR R0, R3, 0                             [line: 259]
           x3259: xF024  PUTSP                                     [line: 260]
           x325A: x1920  ADD R4, R4, #0                            [line: 264]
           x325B: x0808  BRn STAT_NEG (or 8)                       [line: 265]
           x325C: xE19A  LEA R0, STAT_T2                           [line: 267]
           x325D: xF024  PUTSP                                     [line: 268]
           x325E: x38B1  ST R4, CONVERT_VAL                        [line: 270]
           x325F: xEC82  LEA R6, ASCII_CONVERT                     [line: 271]
           x3260: x4E05  JSR PUSH                                  [line: 272]
           x3261: xE1A5  LEA R0, STAT_T4                           [line: 274]
           x3262: xF024  PUTSP 

In [166]:
%dis 32B9

Memory disassembled:
           x32B9: xF024  PUTSP                                     [line: 422]
           x32BA: xF025  HALT                                      [line: 423]
MESSAGE:   x32BB: x1064  ADD R0, R1, #4                            [line: 426]
           x32BC: xF024  PUTSP                                     [line: 427]
           x32BD: x103F  ADD R0, R0, #-1                           [line: 428]
           x32BE: x6000  LDR R0, R0, 0                             [line: 429]
           x32BF: x3175  ST R0, LOC                                [line: 430]
           x32C0: x0E00  BRnzp LOOP_DONE (or 0)                    [line: 431]
LOOP_DONE: x32C1: x0F7D  BRnzp LOOP_START                          [line: 434]
           x32C2: xF025  HALT                                      [line: 435]
PUSH_IND1: x32C3: x3066  ST R0, x332A                              [line: 440]
POP_IND1:  x32C4: x3077  ST R0, DONE                               [line: 441]
CHOICE:    x32C5: xF023  IN    

In [167]:
%dis 331D

Memory disassembled:
           x331D: x2067  LD R0, REMAINDER                          [line: 592]
           x331E: x3023  ST R0, R                                  [line: 593]
           x331F: x2021  LD R0, X                                  [line: 595]
           x3320: x2222  LD R1, Q                                  [line: 596]
           x3321: xEC40  LEA R6, DIVIDE                            [line: 598]
           x3322: x25A0  LD R2, PUSH_IND1                          [line: 599]
           x3323: x4080  JSRR R2                                   [line: 600]
           x3324: x225F  LD R1, QUOTIENT                           [line: 602]
           x3325: x321F  ST R1, TEMP2                              [line: 603]
           x3326: x225E  LD R1, REMAINDER                          [line: 604]
           x3327: x321C  ST R1, TEMP1                              [line: 605]
           x3328: x2016  LD R0, A                                  [line: 607]
           x3329: xEC1D  LEA R6